# Chapter 6: Precognition (Thinking Step by Step)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Define the model to use
modelId = 'anthropic.claude-3-5-haiku-20241022-v1:0'
#modelId = 'anthropic.claude-3-5-sonnet-20241022-v2:0'

session = boto3.Session()
region = session.region_name

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [2]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Claude is the same way.

**Giving Claude time to think step by step sometimes makes Claude more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. You cannot ask Claude to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

In the prompt below, it's clear to a human reader that the second sentence belies the first. But **Claude takes the word "unrelated" too literally**.

In [3]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

The sentiment of this movie review is positive. The phrase "blew my mind with its freshness and originality" is a strong positive statement indicating the reviewer was very impressed with the movie. The humorous follow-up line about living under a rock since 1900 does not change the positive sentiment of the initial review.


To improve Claude's response, let's **allow Claude to think things out first before answering**. We do that by literally spelling out the steps that Claude should take in order to process and think through its task. Along with a dash of role prompting, this empowers Claude to understand the review more deeply.

In [4]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

<positive-argument>
The review suggests enthusiasm with the phrase "blew my mind" and praises the film's "freshness and originality", which are strong positive descriptors indicating the reviewer was impressed by the movie.
</positive-argument>

<negative-argument>
The sarcastic follow-up line about "living under a rock since 1900" implies that the movie might actually not be original at all, and the reviewer is mockingly suggesting that only someone completely disconnected from culture would find this film novel. This suggests the review is actually negative and satirical.
</negative-argument>

Sentiment Analysis: Negative

The review is actually a satirical critique disguised as initial praise. The second sentence reveals the true sentiment, suggesting that the movie is so unoriginal that only someone completely out of touch would find it fresh. The "living under a rock" comment is a clear indication that the reviewer thinks the film is derivative


**Claude is sometimes sensitive to ordering**. This example is on the frontier of Claude's ability to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this changes Claude's overall assessment to positive.

In most situations (but not all, confusingly enough), **Claude is more likely to choose the second of two options**, possibly because in its training data from the web, second options were more likely to be correct.

In [5]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

<negative-argument>
- The review seems sarcastic or ironic
- The phrase "living under a rock" suggests the reviewer might not be credible
- The statement about being disconnected from modern times undermines the praise
</negative-argument>

<positive-argument>
- The direct statement "blew my mind" indicates strong positive sentiment
- The praise of "freshness and originality" suggests high enthusiasm
- The review uses explicitly positive language about the movie
</positive-argument>

Sentiment: Positive (though with a humorous, self-deprecating twist)

The core sentiment is positive, with the reviewer expressing genuine excitement about the movie's qualities. The "living under a rock" comment adds a comedic element but doesn't negate the fundamentally positive review.


**Letting Claude think can shift Claude's answer from incorrect to correct**. It's that simple in many cases where Claude makes mistakes!

Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.

In [6]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

One famous movie starring an actor born in 1956 is "Top Gun" (1986), which stars Tom Cruise.


Let's fix this by asking Claude to think step by step, this time in `<brainstorm>` tags.

In [7]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

<brainstorm>
Tom Cruise - born 1962
Johnny Depp - born 1963
Brad Pitt - born 1963
George Clooney - born 1961
Kevin Bacon - born 1958
John Travolta - born 1954
Nicolas Cage - born 1964
Jeff Goldblum - born 1952
Jim Carrey - born 1962
Will Smith - born 1968
Bruce Willis - born 1955
</brainstorm>

From my brainstorming, I see that Bruce Willis was born in 1955, which is close to the specified year of 1956. A famous movie starring Bruce Willis is "Die Hard" (1988), where he plays John McClane, a New York City police officer who fights terrorists in a Los Angeles skyscraper.


If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
In this exercise, we'll be instructing Claude to sort emails into the following categories:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)

For the first part of the exercise, change the `PROMPT` to **make Claude output the correct classification and ONLY the classification**. Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**.

Refer to the comments beside each email in the `EMAILS` list to know which category that email should be classified under.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email: {email}. Think step-by-step before making the decision\n - First read the email carefully to determine
    its intent.\n - Then categorize the email as per the given categories.\n - Make sure to return only the category and follow the pattern given in the <category_pattern> tag.
    \n\n<category_pattern>Include the letter (A-D) with perentheses as well as the name of the category</category_pattern>"""

# Prefill for Claude's response, if any
PREFILL = "You are an expert email classifier who categorizes emails for below categories:\n - (A) Pre-sale question\n - (B) Broken or defective item\n - (C) Billing question\n - (D) Other (please explain)"

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [9]:
print(hints.exercise_6_1_hint)

The grading function in this exercise is looking for the correct categorization letter + the closing parentheses and the first letter of the name of the category, such as "C) B" or "B) B" etc.
Let's take this exercise step by step:										
1.	How will Claude know what categories you want to use? Tell it! Include the four categories you want directly in the prompt. Be sure to include the parenthetical letters as well for easy classification. Feel free to use XML tags to organize your prompt and make clear to Claude where the categories begin and end.									
2.	Try to cut down on superfluous text so that Claude immediately answers with the classification and ONLY the classification. There are several ways to do this, from speaking for Claude (providing anything from the beginning of the sentence to a single open parenthesis so that Claude knows you want the parenthetical letter as the first part of the answer) to telling Claude that you want the classification and only the classifica

Still stuck? Run the cell below for an example solution.						

In [ ]:
print(hints.exercise_6_1_solution)

### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use your favorite output formatting technique to make Claude wrap JUST the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

In [14]:
# Prompt template with a placeholder for the variable content
PROMPT = """
    You are an expert email classifier who categorizes emails for below categories:\n - (A) Pre-sale question\n - (B) Broken or defective item\n - (C) Billing question\n - (D) Other (please explain)

    Please classify this email: {email}.\n
    Make sure to return the correct choice's letter wrapped in <answer> tag."""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN

    You are an expert email classifier who categorizes emails for below categories:
 - (A) Pre-sale question
 - (B) Broken or defective item
 - (C) Billing question
 - (D) Other (please explain)

    Please classify this email: Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement..

    Make sure to return the correct choice's letter wrapped in <answer> tag.

ASSISTANT TURN


------------------------------------- Claude's response -------------------------------------
Based on the email content, this appears to be a report of a defective or malfunctioning product. The customer is experiencing unusual symptoms with their Mixmaster4000, including a strange noise and a burning smell, which suggests a potential product defect.

<answer>B</answer>

---------------------------------

❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_6_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))